In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import threading

In [2]:
def prepare_training_batch(directory, filenames, n_notes=10):
    train_batch = []
    expected_batch = []

    for i in range(len(filenames)-1):
        first_frame = np.load(directory+filenames[i])
        second_frame = np.load(directory+filenames[i+1])

        frame_length = first_frame.shape[0]
        frames = np.append(first_frame, second_frame, axis=0)

        for j in range(frame_length-1):
            print(f"Interpolating {filenames[i]} to {filenames[i-1]}, on {j} out of {frame_length-1}")

            interpolated_frame = frames[j:j+frame_length+1]
            
            midi_image = []
            for frequency, values in enumerate(np.array(interpolated_frame).transpose()):
                midi_note = int(np.clip(12*math.log((frequency if frequency > 0 else 1)/440, 2)+69, 0, 127))
                while midi_note >= len(midi_image):
                    midi_image.append(np.zeros(len(values)))
                midi_image[midi_note] = np.average(np.array([midi_image[midi_note], values]), axis=0)

            top_n_notes = []
            for slice in np.array(midi_image).transpose():
                new_slice = np.zeros(len(slice))
                for note in np.argsort(slice)[-n_notes:]:
                    new_slice[note] = slice[note]
                top_n_notes.append(new_slice)
            top_n_notes = np.array(top_n_notes)

            # given_tensor = tf.constant(top_n_notes[:-1], dtype=tf.float16)
            # expected_tensor = tf.constant(top_n_notes[-1], dtype=tf.float16)
            
            train_batch.append(top_n_notes[:-1])
            expected_batch.append(top_n_notes[-1])

    return tf.constant(np.array(train_batch), dtype=tf.float16), tf.constant(np.array(expected_batch), dtype=tf.float16)

def prepare_training_data(song_name, batch_size=4, n_notes=10):
    directory = f"./data/fft/{song_name}/"
    if not os.path.isdir(directory): return

    filenames = os.listdir(directory)
    if (".DS_Store" in filenames): filenames.remove(".DS_Store")
    filenames.sort(key=(lambda element : int(element.split("-")[0])))

    all_batch_files = []
    index = 0
    while index + batch_size < len(filenames):
        all_batch_files.append(filenames[index:index+batch_size])
        index += batch_size - 1
    if index < len(filenames):
        all_batch_files.append(filenames[index:])

    training = []
    expected = []

    def create_thread(batch_files):
        this_train, this_expected = prepare_training_batch(directory, batch_files, n_notes)
        training.append(this_train)
        expected.append(this_expected)

    threads = []
    for batch_files in all_batch_files:
        print(f"Creating thread {len(threads)}")
        threads.append(threading.Thread(target=create_thread, args=[batch_files]))
        threads[-1].start()

    for i in range(len(threads)):
        threads[i].join()
        print(f"Joining thread_{i}")

    return training, expected

In [3]:
train, expected = prepare_training_data("test_song", 4, 10)

Creating thread 0
Creating thread 1
Creating thread 2
Creating thread 3
Creating thread 4
Creating thread 5
Creating thread 6
Interpolating 1440000-1920000.npy to 2880000-3360000.npy, on 0 out of 999
Interpolating 2880000-3360000.npy to 4320000-4800000.npy, on 0 out of 999
Interpolating 0-480000.npy to 1440000-1920000.npy, on 0 out of 999
Interpolating 5760000-6240000.npy to 7200000-7680000.npy, on 0 out of 999
Interpolating 8640000-9120000.npy to 9600000-10080000.npy, on 0 out of 999
Interpolating 4320000-4800000.npy to 5760000-6240000.npy, on 0 out of 999
Interpolating 7200000-7680000.npy to 8640000-9120000.npy, on 0 out of 999
Interpolating 1440000-1920000.npy to 2880000-3360000.npy, on 1 out of 999
Interpolating 8640000-9120000.npy to 9600000-10080000.npy, on 1 out of 999
Interpolating 0-480000.npy to 1440000-1920000.npy, on 1 out of 999
Interpolating 2880000-3360000.npy to 4320000-4800000.npy, on 1 out of 999
Interpolating 4320000-4800000.npy to 5760000-6240000.npy, on 1 out of 99

2022-06-27 17:19:22.925655: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-06-27 17:19:22.926411: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Interpolating 480000-960000.npy to 0-480000.npy, on 971 out of 999
Interpolating 4800000-5280000.npy to 4320000-4800000.npy, on 964 out of 999
Interpolating 6240000-6720000.npy to 5760000-6240000.npy, on 988 out of 999
Interpolating 1920000-2400000.npy to 1440000-1920000.npy, on 964 out of 999
Interpolating 3360000-3840000.npy to 2880000-3360000.npy, on 967 out of 999
Interpolating 8160000-8640000.npy to 7680000-8160000.npy, on 0 out of 999
Interpolating 480000-960000.npy to 0-480000.npy, on 972 out of 999
Interpolating 6240000-6720000.npy to 5760000-6240000.npy, on 989 out of 999
Interpolating 4800000-5280000.npy to 4320000-4800000.npy, on 965 out of 999
Interpolating 1920000-2400000.npy to 1440000-1920000.npy, on 965 out of 999
Interpolating 3360000-3840000.npy to 2880000-3360000.npy, on 968 out of 999
Interpolating 8160000-8640000.npy to 7680000-8160000.npy, on 1 out of 999
Interpolating 480000-960000.npy to 0-480000.npy, on 973 out of 999
Interpolating 4800000-5280000.npy to 432000